In [ ]:
import numpy as np
import pandas as pd

# Lendo o arquivo de entrada e preparando a tabela inicial

In [2]:
def ler_entrada(arquivo):
    try:
        with open(arquivo, 'r') as f:
            linhas = [linha.strip() for linha in f.readlines()]
        
        if len(linhas) < 5:
            raise ValueError("O arquivo deve ter pelo menos 5 linhas: número de origens, número de destinos, capacidades de atendimento de cada origem, demandas de cada destino, e matriz de custos de transporte.")
        
        try:
            num_origens = int(linhas[0])
            if num_origens <= 0:
                raise ValueError("O número de origens deve ser um inteiro positivo.")
        except ValueError:
            raise ValueError("A primeira linha deve ser um número inteiro positivo.")

        try:
            num_destinos = int(linhas[1])
            if num_destinos <= 0:
                raise ValueError("O número de destinos deve ser um inteiro positivo.")
        except ValueError:
            raise ValueError("A segunda linha deve ser um número inteiro positivo indicando o número de destinos.")
        
        try:
            cap_atendimento = linhas[2].replace(', ', ' ').split()
            cap_atendimento = [int(x) for x in cap_atendimento]
            if len(cap_atendimento) != num_origens:
                print(f"O número de origens é {num_origens}.")
                print(f"O número de cap é {len(cap_atendimento)}.")
                raise ValueError("A capacidade de atendimento deve ser um vetor com pelo menos 1 posição.")
        except ValueError:
            raise ValueError("A terceira linha deve ser um vetor com pelo menos 1 posição.")
        
        try:
            demandas = linhas[3].replace(', ', ' ').split()
            demandas = [int(x) for x in demandas]
            if len(demandas) != num_destinos:
                print(f"O número de demandas é {len(demandas)}.")
                print(f"O número de destinos é {num_destinos}.")
                raise ValueError(f"A quantidade de demandas deve ser igual ao número de destinos ({num_destinos}).")
        except ValueError:
            raise ValueError("A quarta linha deve ser um vetor com pelo menos 1 posição.")

        if len(linhas) != 4 + num_origens:
            raise ValueError(f"O arquivo deve ter exatamente {4 + num_origens} linhas, mas possui {len(linhas)}.")
        
        try:
            matriz_custos = np.zeros((num_origens, num_destinos), dtype=np.float64)
            for i in range(num_origens):
                valores = linhas[4 + i].replace(', ', ' ').split()
                if len(valores) != num_destinos:
                    raise ValueError(f"A linha {5 + i} deve conter exatamente {num_destinos} valores.")
                matriz_custos[i, :] = [float(x) for x in valores]
        except ValueError:
            raise ValueError("A quarta linha deve conter os coeficientes da função objetivo, separados por vírgulas ou espaços.")
        
        return num_origens, num_destinos, cap_atendimento, demandas, matriz_custos

    except FileNotFoundError:
        raise FileNotFoundError(f"O arquivo '{arquivo}' não foi encontrado.")
    except Exception as e:
        raise RuntimeError(f"Erro ao ler o arquivo: {e}")

In [3]:
# Teste de leitura e preparação da tabela
arquivo = 'Entrada2.txt'
num_origens, num_destinos, cap_atendimento, demandas, matriz_custos = ler_entrada(arquivo)

print(f"Número de origens: {num_origens}")
print(f"Número de destinos: {num_destinos}")
print(f"Demandas: {demandas}")
print(f"Capacidades de atendimento: {cap_atendimento}")
print("Matriz de custos:")
print(matriz_custos)

Número de origens: 3
Número de destinos: 4
Demandas: [5, 15, 15, 15]
Capacidades de atendimento: [15, 25, 10]
Matriz de custos:
[[10.  2. 20. 11.]
 [12.  7.  9. 20.]
 [ 4. 14. 16. 18.]]


In [4]:
def balancear_problema(num_origens, num_destinos, cap_atendimento, demandas):
    if sum(cap_atendimento) > sum(demandas):
        print("Capacidade de atendimento é maior que a demanda total")
        demandas.append(sum(cap_atendimento) - sum(demandas))
        num_destinos += 1
    elif sum(cap_atendimento) < sum(demandas):
        print("Demanda total é maior que a capacidade de atendimento")
        cap_atendimento.append(sum(demandas) - sum(cap_atendimento))
        num_origens += 1
    return num_origens, num_destinos, cap_atendimento, demandas

In [5]:
num_origens, num_destinos, cap_atendimento, demandas = balancear_problema(num_origens, num_destinos, cap_atendimento, demandas)

print(f"Número de origens: {num_origens}")
print(f"Número de destinos: {num_destinos}")
print(f"Demandas: {demandas}")
print(f"Capacidades de atendimento: {cap_atendimento}")
print("Matriz de custos:")
print(matriz_custos)

Número de origens: 3
Número de destinos: 4
Demandas: [5, 15, 15, 15]
Capacidades de atendimento: [15, 25, 10]
Matriz de custos:
[[10.  2. 20. 11.]
 [12.  7.  9. 20.]
 [ 4. 14. 16. 18.]]


In [ ]:
def gerar_tabela_inicial(num_origens, num_destinos, cap_atendimento, demandas, matriz_custos):    
    fabricas = [f"O{i+1}" for i in range(num_origens)]
    distribuicao = [f"D{j+1}" for j in range(num_destinos)]
    
    tabela = pd.DataFrame(matriz_custos, index=fabricas, columns=distribuicao)
    tabela["Oferta"] = cap_atendimento
    demanda_row = pd.DataFrame([demandas + [None]], index=["Demanda"], columns=tabela.columns)
    
    tabela = pd.concat([tabela, demanda_row])
    
    return tabela

tabela = gerar_tabela_inicial(num_origens, num_destinos, cap_atendimento, demandas, matriz_custos)
print("Tabela inicial:")
tabela

Tabela inicial:


,D1,D2,D3,D4,Oferta
O1,10.0,2.0,20.0,11.0,15
O2,12.0,7.0,9.0,20.0,25
O3,4.0,14.0,16.0,18.0,10
Demanda,5.0,15.0,15.0,15.0,None


In [ ]:
def gerar_solucao_basica_viavel(tabela):
    solucao = tabela.copy()

    # Matriz para armazenar as alocações sem modificar os custos
    alocacoes = pd.DataFrame(np.zeros_like(solucao.iloc[:-1, :-1]), 
                             index=solucao.index[:-1], 
                             columns=solucao.columns[:-1])

    contador_linha = 0
    contador_coluna = 0
    
    while solucao.iloc[:-1, -1].sum() > 0 and solucao.iloc[-1, :-1].sum() > 0:
        oferta_disponivel = solucao.iloc[contador_linha, -1]
        demanda_disponivel = solucao.iloc[-1, contador_coluna]
        
        # Alocar o mínimo entre a oferta e a demanda (máximo possível para a célula)
        quantidade = min(oferta_disponivel, demanda_disponivel)
        alocacoes.iloc[contador_linha, contador_coluna] = quantidade

        # Atualizar oferta e demanda
        solucao.iloc[contador_linha, -1] -= quantidade
        solucao.iloc[-1, contador_coluna] -= quantidade

        if solucao.iloc[contador_linha, -1] == 0:
            contador_linha += 1
        elif solucao.iloc[-1, contador_coluna] == 0:
            contador_coluna += 1

    return alocacoes

solucao = gerar_solucao_basica_viavel(tabela)
solucao

,D1,D2,D3,D4
O1,5.0,10.0,0.0,0.0
O2,0.0,5.0,15.0,5.0
O3,0.0,0.0,0.0,10.0
